### 1. LeNet5 Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import numpy as np

class ScaledTanh(nn.Module):
    def forward(self, x):
        return 1.7159 * torch.tanh(x * 2 / 3)
    
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.tanh = ScaledTanh()

        # C1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        
        # S2
        self.weight2 = nn.Parameter(torch.ones(1, 6, 1, 1))
        self.bias2 = nn.Parameter(torch.zeros(1, 6, 1, 1))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight2)
        nn.init.uniform_(self.weight2, -2.4 / fan_in, 2.4 / fan_in)
        self.bias2.data.fill_(2.4 / fan_in)

        # C3
        self.conv3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        mask = torch.zeros_like(self.conv3.weight, dtype=torch.bool)
        table = self.connection_table()
        for out_idx, conn in enumerate(table):
            mask[out_idx, conn] = True
        self.register_buffer("conv3_mask", mask.float())
        with torch.no_grad():
            self.conv3.weight *= self.conv3_mask

        # S4
        self.weight4 = nn.Parameter(torch.ones(1, 16, 1, 1))
        self.bias4 = nn.Parameter(torch.zeros(1, 16, 1, 1))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight4)
        nn.init.uniform_(self.weight4, -2.4 / fan_in, 2.4 / fan_in)
        self.bias4.data.fill_(2.4 / fan_in)

        # C5
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1)

        # F6
        self.fc6 = nn.Linear(120, 84)

        # Output Layer
        self.prototypes = self.compute_rbf_prototypes()

    def connection_table(self):
        return [
            [0, 1, 2],
            [1, 2, 3],
            [2, 3, 4],
            [3, 4, 5],
            [0, 4, 5],
            [0, 1, 5],
            [0, 1, 2, 3],
            [1, 2, 3, 4],
            [2, 3, 4, 5],
            [0, 3, 4, 5],
            [0, 1, 4, 5],
            [0, 1, 2, 5],
            [0, 1, 3, 4],
            [1, 2, 4, 5],
            [1, 2, 3, 5],
            [0, 1, 2, 3, 4, 5]
        ]

    def compute_rbf_prototypes(self):
        import matplotlib.pyplot as plt

        prototypes = []

        image_folder = './digits updated/'
        bitmap_size = (7,12)
        num_classes = 10

        for label in range(num_classes):
            class_folder = os.path.join(image_folder, str(label))
            images = []
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                image = cv2.imread(img_path, 0)
                if image is not None:
                    image = cv2.resize(image, bitmap_size)
                    image = 255.0 - image  # Invert colors
                    image = (image > 127).astype(np.float32)  # Binarize to 0 and 1
                    image = image / 255.0

                    images.append(image)
            if images:
                mean_image = np.mean(images, axis=0)
                prototypes.append(mean_image.flatten())

        prototypes_arr = np.array(prototypes)

        return torch.tensor(prototypes_arr, dtype=torch.float32)

    def compute_rbf_distance(self, x):
        x = (x - x.mean(dim=1, keepdim=True)) / (x.std(dim=1, keepdim=True) + 1e-5)
        prototypes = (self.prototypes - self.prototypes.mean(dim=1, keepdim=True)) / (self.prototypes.std(dim=1, keepdim=True) + 1e-5)

        # L2 normalize input features and prototypes
        x = F.normalize(x, p=2, dim=1)  # shape: [batch_size, feature_dim]
        prototypes = F.normalize(prototypes, p=2, dim=1)  # shape: [num_classes, feature_dim]

        # Compute pairwise squared Euclidean distances
        x_expanded = x.unsqueeze(1)  # shape: [batch_size, 1, feature_dim]
        prototypes_expanded = prototypes.unsqueeze(0)  # shape: [1, num_classes, feature_dim]
        output = (x_expanded - prototypes_expanded).pow(2).sum(-1)  # shape: [batch_size, num_classes]

        return output   

    def forward(self, x):
        # C1
        x = self.conv1(x)
        x = self.tanh(x)

        # S2
        x = F.avg_pool2d(x, kernel_size=2, stride=2) * self.weight2.view(1, -1, 1, 1) + self.bias2.view(1, -1, 1, 1)
        x = self.tanh(x)

        # C3
        self.conv3.weight.data *= self.conv3_mask  # Apply the mask to the weights
        x = self.conv3(x)
        x = self.tanh(x)

        # S4
        x = F.avg_pool2d(x, kernel_size=2, stride=2) * self.weight4.view(1, -1, 1, 1) + self.bias4.view(1, -1, 1, 1)
        x = self.tanh(x)

        # C5
        x = self.conv5(x)
        x = self.tanh(x)

        # F6
        x = x.view(x.size(0), -1)
        x = self.fc6(x)

        # Output Layer
        x = self.compute_rbf_distance(x)
        return x

### 2. Load Train and Test Data

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms
from PIL import Image
train_image_folder = './data/train/'
test_image_folder = './data/test/'
train_label_file = './data/train_label.txt'
test_label_file = './data/test_label.txt'

train_images = []
train_labels = []
test_images = []
test_labels = []

transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 32x32
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

with open(train_label_file, 'r') as f:
    label_lines = f.readlines()
    image_filenames = sorted(os.listdir(train_image_folder))

    for idx in range(len(label_lines)):
        img_name = f"{idx}.png"
        img_path = os.path.join(train_image_folder, img_name)
        img = cv2.imread(img_path, 0)
        if img is not None:
            image = Image.fromarray(img)
            image = transform(image)
            train_images.append(image)
            label = int(label_lines[idx].strip())
            train_labels.append(label)

with open(test_label_file, 'r') as f:
    label_lines = f.readlines()
    image_filenames = sorted(os.listdir(train_image_folder))

    for idx in range(len(label_lines)):
        img_name = f"{idx}.png"
        img_path = os.path.join(test_image_folder, img_name)
        img = cv2.imread(img_path, 0)
        if img is not None:
            image = Image.fromarray(img)
            image = transform(image)
            test_images.append(image)
            label = int(label_lines[idx].strip())
            test_labels.append(label)

train_images = torch.stack(train_images)
test_images = torch.stack(test_images)
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

In [ ]:
train_loader = DataLoader(TensorDataset(train_images, train_labels), batch_size=1, shuffle=False)
test_loader = DataLoader(TensorDataset(test_images, test_labels), batch_size=1, shuffle=False)

### 3. Train the Model

In [ ]:
def customLoss(outputs, labels, j=0.1):
    batch_size = outputs.size(0)

    # Correct class distances
    pos_dists = outputs[torch.arange(batch_size), labels]  # Shape: [B]

    # Mask out correct class
    mask = torch.ones_like(outputs, dtype=torch.bool)
    mask[torch.arange(batch_size), labels] = False
    neg_dists = outputs[mask].view(batch_size, -1)  # Shape: [B, C-1]

    # Stable discriminative log-sum-exp term
    # log(e^{-j} + sum(e^{-d_i})) = logsumexp([-j, -d1, -d2, ..., -d9])
    margin_tensor = torch.full((batch_size, 1), -j, device=outputs.device)
    all_terms = torch.cat([margin_tensor, -neg_dists], dim=1)
    log_term = torch.logsumexp(all_terms, dim=1)

    # Final loss (lower distance is better)
    loss = (-pos_dists + log_term).mean()
    return loss

In [186]:
from tqdm import tqdm

# Define relevant variables
num_classes = 10
learning_rate = 0.001
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet5(num_classes=num_classes).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

total_step = len(train_loader)

# Initialize lists to store accuracy and error rates for plotting
train_accuracy_list = []
test_accuracy_list = []
train_loss_list = []
test_loss_list = []

for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    train_loss = 0.0
    for i, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = customLoss(outputs, labels)
        train_loss_list.append(loss.item())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tracking accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total


    # After training the epoch, evaluate on the test set
    model.eval()  # Set the model to evaluation mode
    test_correct = 0
    test_total = 0
    test_loss = 0

    with torch.no_grad():  # No gradient tracking during evaluation
        for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)

            # Compute the loss
            loss = customLoss(outputs, labels)
            test_loss_list.append(loss.item())

            # Compute the accuracy
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    # Testing accuracy and error rate for the epoch
    test_accuracy = 100 * test_correct / test_total

    # Print the results for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Accuracy: {train_accuracy:.2f}%, '
          f'Testing Accuracy: {test_accuracy:.2f}%, ')


    # Store values for plotting later
    train_accuracy_list.append(train_accuracy)
    test_accuracy_list.append(test_accuracy)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)

    # print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Training Accuracy: {:.2f}%'.format(epoch+1, num_epochs, i+1, total_step, loss.item(), accuracy))

    # accuracy = 100 * correct / total
    # print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {accuracy:.2f}%')

Epoch [1/20], Training Accuracy: 20.05%, Training Loss: -211172.80%, Testing Accuracy: 60.81%, Testing Loss: -36308.61%


Epoch [2/20], Training Accuracy: 73.70%, Training Loss: -219410.00%, Testing Accuracy: 75.60%, Testing Loss: -36723.34%


Epoch [3/20], Training Accuracy: 80.46%, Training Loss: -220932.91%, Testing Accuracy: 82.03%, Testing Loss: -36932.90%


Epoch [4/20], Training Accuracy: 85.96%, Training Loss: -222037.30%, Testing Accuracy: 88.37%, Testing Loss: -37082.37%


Epoch [5/20], Training Accuracy: 90.33%, Training Loss: -222883.52%, Testing Accuracy: 90.79%, Testing Loss: -37178.09%


...

Epoch [16/20], Training Accuracy: 95.48%, Training Loss: -224554.21%, Testing Accuracy: 95.30%, Testing Loss: -37427.73%


Epoch [17/20], Training Accuracy: 95.64%, Training Loss: -224628.36%, Testing Accuracy: 95.56%, Testing Loss: -37440.73%


Epoch [18/20], Training Accuracy: 95.79%, Training Loss: -224698.48%, Testing Accuracy: 95.78%, Testing Loss: -37452.97%


Epoch [19/20], Training Accuracy: 95.95%, Training Loss: -224763.84%, Testing Accuracy: 95.96%, Testing Loss: -37464.32%


Epoch [20/20], Training Accuracy: 96.07%, Training Loss: -224824.32%, Testing Accuracy: 96.07%, Testing Loss: -37474.75%


In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for efficiency
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

### 4. Plot the Results

In [ ]:
# After training is done, you can plot the results
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(range(1, num_epochs+1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs+1), test_accuracies, label='Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Training and Testing Accuracy')

# Plot error rate
plt.figure()
plt.plot(range(1, num_epochs+1), train_error_rates, label='Training Error Rate')
plt.plot(range(1, num_epochs+1), test_error_rates, label='Testing Error Rate')
plt.xlabel('Epochs')
plt.ylabel('Error Rate (%)')
plt.legend()
plt.title('Training and Testing Error Rate')

# Show the plots
plt.show()

### 5. Save the Model

In [ ]:
    # torch.save(model.state_dict(), 'model_epoch_{}.pth'.format(epoch+1))
